In [1]:
%matplotlib ipympl

import os
import numpy as np
import matplotlib.pyplot as plt
import utide
import matplotlib.dates as mdates
import gsw
from physoce import tseries as ts
import otis_tide_pred as otp
from ADCP import rditext_to_dataset

In [2]:
data_dir = '/Users/tomconnolly/work/Data/MBARI_Station_M/ADCP'
data_filename = 'MBARI_StationM_ADCP_201711_201811.txt'
data_path = os.path.join(data_dir,data_filename)

In [3]:
ds = rditext_to_dataset(data_path)

t1 = np.datetime64('2017-11-10 02:00')
t2 = np.datetime64('2018-10-17 02:00')

ii, = np.where((ds['time'] >= t1) & (ds['time'] <= t2))
ds = ds.isel(time=ii)

ds['Eas'] = ds['Eas']/1000
ds['Nor'] = ds['Nor']/1000

In [4]:
ds

<xarray.Dataset>
Dimensions:    (beam: 4, bin: 23, time: 98152)
Coordinates:
  * time       (time) datetime64[ns] 2017-11-10T06:43:13.760000 ...
Dimensions without coordinates: beam, bin
Data variables:
    Pit        (time) float64 0.07 -25.98 -25.98 -25.98 -25.98 -25.98 -25.98 ...
    Rol        (time) float64 0.58 24.25 24.25 24.25 24.25 24.25 24.25 24.25 ...
    Hea        (time) float64 30.85 346.9 350.9 348.3 347.2 329.2 317.6 ...
    Tem        (time) float64 19.46 19.48 19.42 16.59 15.35 11.79 9.43 7.47 ...
    Dep        (time) float64 4e+03 4e+03 4e+03 4e+03 4e+03 4e+03 4e+03 ...
    Ori        (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...
    BIT        (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    Bat        (time) int64 178 126 126 119 119 119 120 119 119 119 119 119 ...
    Eas        (time, bin) float64 nan nan nan nan nan nan nan nan nan nan ...
    Nor        (time, bin) float64 nan nan nan nan nan nan nan nan nan nan ...
    V

In [5]:
plt.figure()
plt.plot(ds['time'],ds['Eas'][:,:15]);

FigureCanvasNbAgg()

In [6]:
bini = np.arange(1,17)

plt.figure()
plt.plot(np.mean(ds['Eas'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.plot(np.mean(ds['Nor'][:,bini],axis=0),ds['binheight'][bini],'-o')
plt.title('mean velocity')
plt.legend(['East','North'])

FigureCanvasNbAgg()

In [7]:
plt.figure()
plt.plot(ds['Eas'][:,1],ds['Nor'][:,1],'.')
plt.gca().axis('equal')

FigureCanvasNbAgg()

(-0.000243, 0.000175, -0.00024565, 0.00023065)

In [8]:
plt.figure()
plt.plot(ds['time'],ds['C'][:,:,3]);

FigureCanvasNbAgg()

In [9]:
time = mdates.date2num(ds['time'])

coef = utide.solve(time, ds['Eas'][:,1], ds['Nor'][:,1],
             lat=34+50/60)

solve: matrix prep ... solution ... 

/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/UTide-0.1.0-py3.5.egg/utide/_solve.py:289: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  m = np.linalg.lstsq(B, xraw)[0]  # Model coefficients.


diagnostics ... done.


In [10]:
tide = utide.reconstruct(time, coef)

prep/calcs ... done.


In [11]:
tide

{'u': array([-5.33742025e-06, -5.82796280e-06, -6.32495646e-06, ...,
         1.65722950e-05,  1.65912091e-05,  1.66122442e-05]),
 'v': array([-3.41322213e-05, -3.45735466e-05, -3.49427867e-05, ...,
         2.21178267e-05,  2.21119111e-05,  2.20745249e-05])}

In [12]:
plt.figure()
plt.plot(ds['time'],tide['u'])
plt.plot(ds['time'],tide['v'])

FigureCanvasNbAgg()

In [13]:
plt.figure()
plt.plot(tide['u'],tide['v'],'.')
plt.gca().axis('equal')

FigureCanvasNbAgg()

(-5.949796821791919e-05,
 4.371085844772233e-05,
 -6.210519676163516e-05,
 5.559194512507214e-05)

In [14]:
plt.figure()
plt.plot(ds['time'],ds['Eas'][:,1]-tide['u'])
plt.plot(ds['time'],ds['Nor'][:,1]-tide['v'])

FigureCanvasNbAgg()

In [15]:
ds['Eas_filt'] = (['time','bin'],ts.pl66(ds['Eas'],T=33*12))
ds['Nor_filt'] = (['time','bin'],ts.pl66(ds['Nor'],T=33*12))

/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/scipy/signal/signaltools.py:459: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return correlate(volume, kernel[slice_obj], mode)
/Users/tomconnolly/programs/anaconda/lib/python3.5/site-packages/scipy/signal/signaltools.py:179: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  in1zpadded[sc] = in1.copy()


In [16]:
plt.figure()
plt.plot(ds['time'],ds['Eas_filt'][:,bini],'r-')
plt.plot(ds['time'],ds['Nor_filt'][:,bini],'b-')
plt.plot(ds['time'],np.zeros(len(ds['time'])),'k--')

FigureCanvasNbAgg()

In [17]:
# intertial period (days)
T = 2*np.pi/gsw.f(34+50/60)/86400
T

0.8729736117491941

### Future processing steps
* Fill small gaps
* QC based on correlation, percent good, etc.

### Ideas for future analysis

* Wavelet analysis: isolate time variability of inertial band
* Relate inertial currents to local wind
* Complex correlation of low-pass filtered currents
* Is low frequency variability due to eddies? Relate to satellite altimeter data
* Explore relationship to local and remote wind stress curl
* Read Clarke paper on ("too fast") Rossby wave propagation in CalCOFI region
* Compare tidal currents to OSU model
* Progressive vector diagram